# Concrete translation of specification

These are notes on how we will translate the general specification of the optimisation problem into an optimisation problem that our optimiser (Pyomo) can solve using its existing functionality.

# Specification

This is the current problem specification in abstract terms.

## Indices and sets

The index is a label that identifies an element in a set. For simplicity, we shall use the index to refer to the element that it indexes.

* $af \in AF$ is an index for an openLCA product flow imported from an openLCA database. This includes new flows that are defined by the user for the optimisation problem.
* $f\in F$ is an index for a user-defined flow.
* $f_m \in F_m \subset F$ is an index for a user-defined material flow (e.g. energy, material) to be considered in the optimisation problem.
* $f_s \in F_s \subset F$ is an index for an user-defined service flow (e.g. energy storage, transport) to be considered in the optimisation problem.
* $f_{t} \in F_{t} \subset F_s$ is an index for transport service flow i.e. transport mode \{road, train freight, air etc\}.
* $e \in E$ is an elementary flow index for elementary flows imported from an openLCA database. A system process in the openLCA database is by definition broken down into a set of these elementary flows.
* $ap \in AP$ is an index for a process in the set of all processes contained in an openLCA database. This include user-defined processes specifically designed for the optimisation tool.
* $p\in P\subset AP$ is a process index for processes that make up the optimisation problem.
* $t \in T$ is the time interval by time discretization $\{t_1, t_2, t_3, t_4 \ldots t_n\}$.
* $k \in K$ a task index $k$ in the set of all task indices $K$.
* $d \in D$ an index for a demand $d$ in the set of demand indices $D$. 
* $akpi \in AKPI$ is an indexes for all key performance indicators $KPI$ in an openLCA database. This includes key performance indicators defined by the user of the optimisation tool that must be added to the openLCA database.
* $kpi \in KPI\subset AKPI$ is an index that identifies those performance indicators that the user wish to use in the optimisation problem.

## Parameters

### User-defined

* $C_{f_m, k, d, t}$ Conversion factor for material flows to generate per unit of demand product/services $d$ at task $s$, time $t$. If not defined default value is 0.

* $D_{d,t}$ Demand for final product/service $d$ at time t; If not defined, default value is 0.

* $D_d^{total}$ Total demand for final product/service over the whole optimisation time period.

* $L_{f_m, f_s}$ Binary factor to link services flows to material flows e.g. energy storage, material storage, transport; If not defined, default value is 0.

* $d_{f_p, f_t}$ Travel distance of material $f_p$ by travel mode $f_t$.

* $\phi_{f, t}$ Cost co-efficient for material and service flows at time t


### Imported from openLCA and model initialisation

* $Ef_{akpi, e}$ Environmental impact characterisation factor for elementary flow $e$ and performance indicator $akpi$.

* $EF_{e, f, p}$ Elementary flow $e$ to link with product flow $f$ through process $p$.

* $EI_{akpi, f, ap}$ Calculated environmental impact for product flow $𝑓$ through process $ap$ and performance indicator $kpi$.

### Continuous variables

* $Obj_{kpi}$ Objective functions for the user-defined KPIs.
* $Flow_{f_m, k, t}$ is the flow of material input $f_m$ to task $k$ at time $t$.
* $S_{f_s, k, t}$ is the temporary storage of service input flow $f_s$ at task $k$ and time $t$. **Perhaps call it $Service$ like $Flow$.**

## Objective function

Our objective is to minimise the environmental impact of elementary flows and the economic cost derived from a network of processes.

Consequently, the objective is for a fixed impact category $kpi$

$$
\min_{Flow_{f_m, k, t}, S_{f_s, k, t}} Obj_{kpi}
$$

and

$$
\min_{Flow_{f_m, k, t}, S_{f_s, k, t}} Obj_{cost}
$$

The environmental impact is the sum of the environmental impacts arising from material and service flows:

$$
Obj_{kpi} = \sum_{f_m, k, t} Flow_{f_m, k, t}EI_{kpi, f_m} + \sum_{f_s, k ,t} S_{f_s, k, t}EI_{kpi, f_s}.
$$

The economic impact is the sum of the economic impacts arising from material and service flows:

$$
Obj_{cost} = \sum_{f_m, k, t} Flow_{f_m, k, t}\phi_{f_m, t} + \sum_{f_s,k,t} S_{f_s, k,t}\phi_{f_s, t}.
$$

Here the environmental impact of flow $f\in F$ measured by impact factor $kpi$ is 

$$
EI_{kpi, f} = \sum_e Ef_{kpi, e}EF_{e, f, p}
$$

where the flow $f$ is the product flow for the process $p\in P$. Here $Ef_{kpi, e}$ denotes the impact factor indexed by impact category $kpi$ and environmental flow $e$ and $EF_{e, f, p}$ is the amount of elementary flow generated by the product flow $f$ for $p$. If $f\in F_m\cup F_s$ then the breakdown of flow into elementary flow amounts $EF_{e, f, p}$ must be calculated in openLCA by constructing a *system process*, which is then imported into the optimisation tool. Otherwise the flow is a product flow from an existing system process in openLCA so there already is a breakdown.

The quantity of service flow $S_{f_s,k,t}$ is assumed to be a weighted sum of the material flows at task $k$ and time $t$ i.e.

$$
S_{f_s,k,t} = \sum_{f_m} L_{f_m, f_s}Flow_{f_m, k, t}d_{f_m,f_{s\in F_t}},
$$

where $L_{f_m, f_s}$ is a parameter than links the service flow $f_s$ to the material flow $f_m$ in task $k$ at time $t$. In the case of transport of $f_m$ is enabled then $L_{f_m,f_s}=1$ and the variable $S_{f_s,k,t}$ is also
dependent on the transport distance for shipping $f_m$ via transport mode $f_t$.

## Constraints

The conversion of temporary storage to flow and demand requires

$$
Flow_{f_m, k, t} \geq \sum_d D_{d,t} C_{f_m, k, d, t} + S_{f_m, k, t} - S_{f_m, k, t-1}
$$

The total material flow minus final storage must satisfy the total demand over the time horizon so

$$
\sum_t Flow_{f_m,k,t}  \geq \sum_{d,t} D_d^{total} C_{f_m, k, t} + S_{f_m,k,t_n}
$$

Finally, we require positive flows and storage

$$
F_{f_m, k, t} \geq 0
$$

$$
S_{f_s, k, t} \geq 0
$$

## Network of processes and flows

The aggregate flow quantity $EF_{e, f, p}$ is determined by its network of contributing flows. We can identify the set of flows linked to a process indexed by $p\in P$ using the flow index set

$$
Z_p = \{f\in F\ s.t.\ \sum_e EF_{e, f, p} > 0\}.
$$

However, in practice it is impractical for a user to build a network by constructing these sets because there are thousands of elementary flows.


# Problem definition